# 规则引擎设计

## 规则报文

### 联动规则
> 联动规则通过添加策略接口添加
```json
{
  "id": "test1",
  "name": "pedestrianCrossing",
  "description": "pedestrianCrossing",
  "priority": 51,
  "productIdentifier": "S8_RLY_RUILONJGS869",
  "devSn": "dev1",
  "active": "Y",
  "timeout": 0,
  "eventIdentifier": "invade1",
  "conditions": [
    {
      "target": "onOff",
      "operator": "E",
      "comparison": "1"
    },
    {
      "target": "brightness",
      "operator": "E",
      "comparison": "2"
    }
  ],
  "actions": [
    {
      "type": "CALL_SERVICE",
      "productIdentifier": "S5_HKDomeCamera",
      "devSn": "hk22000002",
      "params": "{ \"serviceIdentifier\": \"setOnOff\",\"serviceParams\": {\"onOff\": 1 }}"
    },
    {
      "type": "CALL_SERVICE",
      "productIdentifier": "S3_KLT_C4",
      "devSn": "62711",
      "params": "{ \"serviceIdentifier\": \"setOnOff\",\"serviceParams\": {\"onOFF\": 1 }}"
    }
  ],
  "effectiveDate": {
    "startDate": "2022-09-15",
    "endDate": "2022-12-15",
    "startTime": "00:00:00",
    "endTime": "23:00:00"
  },
  "retryCount": 0
}
```
### 定时规则
>规则引擎固化一条定时策略用于匹配每个设备产生的定时事件
```json
 {
  "actions": [
    {
      "productIdentifier": "${productIdentifier}",
      "devSn": "${devSn}",
      "type": "CALL_SERVICE",
      "params":""
    }
  ],
  "active": "Y",
  "conditions": [],
  "description": "",
  "eventIdentifier": "TIMER_START",
  "execUnit": "CLOUD",
  "identifier": "TIMER_START",
  "name": "设备定时开始事件",
  "priority": 50,
  "productIdentifier": "",
  "retryCount": 0,
  "timeout": 58,
  "effectiveDate": {
    "startDate": "1970-01-01",
    "endDate": "2070-01-01",
    "startTime": "00:00:00",
    "endTime": "23:59:59"
  },
  "id": "TIMER_START_RULE"
}
```
### 临时手动规则
>规则引擎固化一条临时手动策略用于匹配每个设备产生的手动事件

```json
     {
  "actions": [
    {
      "productIdentifier": "${productIdentifier}",
      "devSn": "${devSn}",
      "type": "CALL_SERVICE",
      "params": ""
    }
  ],
  "active": "Y",
  "conditions": [],
  "description": "",
  "eventIdentifier": "MANUAL_SERVICE",
  "execUnit": "CLOUD",
  "identifier": "MANUAL_SERVICE",
  "name": "手动控制",
  "priority": 100,
  "productIdentifier": "",
  "retryCount": 0,
  "timeout": 5,
  "effectiveDate": {
    "startDate": "1970-01-01",
    "endDate": "2070-01-01",
    "startTime": "00:00:00",
    "endTime": "23:59:59"
  },
  "id": "MANUAL_RULE"
}
```
### 外部联动规则
>规则引擎固化一条外部联动规则，用于匹配平台端所产生的事件
```json
 {
  "actions": [
    {
      "productIdentifier": "${productIdentifier}",
      "devSn": "${devSn}",
      "type": "CALL_SERVICE",
      "params": ""
    }
  ],
  "active": "Y",
  "conditions": [],
  "description": "",
  "eventIdentifier": "MANUAL_CONTROL",
  "execUnit": "CLOUD",
  "identifier": "MANUAL_CONTROL",
  "name": "外部控制",
  "priority": 100,
  "productIdentifier": "",
  "retryCount": 0,
  "timeout": 5,
  "effectiveDate": {
    "startDate": "1970-01-01",
    "endDate": "2070-01-01",
    "startTime": "00:00:00",
    "endTime": "23:59:59"
  },
  "id": "EXTERNAL_RULE"
}
```


## 对外服务
>规则引擎提供规则的添加策略、更新策略、删除策略、修改策略、获取策略、启用策略、停用策略、外部事件联动、规则统计服务.

### 添加策略

```json
{
  "id": "addRule",
  "name": "添加策略",
  "parameters": [
    {
      "id": "id",
      "name": "规则ID",
      "type": "string"
    },
    {
      "id": "name",
      "name": "规则名字",
      "type": "string"
    },
    {
      "id": "devSn",
      "name": "触发源设备sn",
      "type": "string"
    },
    {
      "id": "active",
      "name": "停用启用状态",
      "type": "string",
      "specs": {
        "optional": [
          {
            "value": "Y",
            "desc": "启用"
          },
          {
            "value": "N",
            "desc": "停用"
          }
        ]
      }
    },
    {
      "id": "description",
      "name": "规则描述",
      "type": "string"
    },
    {
      "id": "productIdentifier",
      "name": "触发源产品标识符",
      "type": "string"
    },
    {
      "id": "priority",
      "name": "优先级",
      "type": "integer",
      "specs": {
        "min": 0,
        "max": 99
      }
    },
    {
      "id": "timeout",
      "name": "规则动作执行超时时间",
      "type": "integer",
      "specs": {
        "min": -1
      }
    },
    {
      "id": "retryCount",
      "name": "执行动作失败重试次数",
      "type": "integer",
      "specs": {
        "min": 0,
        "max": 10
      }
    },
    {
      "id": "conditions",
      "name": "执行条件，条件是与(并且)关系",
      "type": "array",
      "columnComplex": [
        {
          "id": "target",
          "name": "输入规则的事件参数，暂时不支持数组，结构体层级结构使用英文.作为分隔符",
          "type": "string"
        },
        {
          "id": "operator",
          "name": "判断符",
          "type": "string",
          "specs": {
            "optional": [
              {
                "value": "GT",
                "desc": "大于"
              },
              {
                "value": "GTE",
                "desc": "大于等于"
              },
              {
                "value": "LT",
                "desc": "小于"
              },
              {
                "value": "LTE",
                "desc": "小于等于"
              },
              {
                "value": "E",
                "desc": "等于"
              },
              {
                "value": "NE",
                "desc": "不等于"
              },
              {
                "value": "CONTAIN",
                "desc": "包含"
              },
              {
                "value": "NOT_CONTAIN",
                "desc": "不包含"
              }
            ]
          }
        },
        {
          "id": "comparison",
          "name": "比较值",
          "type": "string"
        }
      ]
    },
    {
      "id": "actions",
      "name": "执行动作",
      "type": "array",
      "columnComplex": [
        {
          "id": "type",
          "name": "执行动作类型",
          "type": "string",
          "specs": {
            "optional": [
              {
                "value": "CALL_SERVICE",
                "desc": "CALL_SERVICE"
              },
              {
                "value": "EVENT",
                "desc": "EVENT"
              },
              {
                "value": "WORKFORM",
                "desc": "WORKFORM"
              },
              {
                "value": "MESSAGE",
                "desc": "MESSAGE"
              },
              {
                "value": "EMAIL",
                "desc": "EMAIL"
              },
              {
                "value": "SLEEP",
                "desc": "SLEEP"
              }
            ]
          }
        },
        {
          "id": "productIdentifier",
          "name": "执行动作的产品标识符",
          "type": "string"
        },
        {
          "id": "devSn",
          "name": "执行动作的设备sn",
          "type": "string"
        },
        {
          "id": "params",
          "name": "执行动作参数，因动作类型不同而异，详情请查看规则json描述文档",
          "type": "string"
        }
      ]
    },
    {
      "id": "effectiveDate",
      "name": "规则有效时间周期",
      "type": "struct",
      "struct": [
        {
          "id": "startDate",
          "name": "开始日期",
          "format": "date",
          "type": "string"
        },
        {
          "id": "endDate",
          "name": "结束日期",
          "format": "date",
          "type": "string"
        },
        {
          "id": "startTime",
          "name": "开始时间",
          "format": "time",
          "type": "string"
        },
        {
          "id": "endTime",
          "name": "结束时间",
          "format": "time",
          "type": "string"
        }
      ]
    },
    {
      "id": "eventIdentifier",
      "name": "事件标识符",
      "type": "string"
    }
  ],
  "output": []
}
```

### 更新策略

```json
{
  "id": "updateRule",
  "name": "更新策略",
  "parameters": [
    {
      "id": "id",
      "name": "规则ID",
      "type": "string"
    },
    {
      "id": "name",
      "name": "规则名字",
      "type": "string"
    },
    {
      "id": "active",
      "name": "停用启用状态",
      "type": "string",
      "specs": {
        "optional": [
          {
            "value": "Y",
            "desc": "启用"
          },
          {
            "value": "N",
            "desc": "停用"
          }
        ]
      }
    },
    {
      "id": "description",
      "name": "规则描述",
      "type": "string"
    },
    {
      "id": "productIdentifier",
      "name": "触发源产品标识符",
      "type": "string"
    },
    {
      "id": "devSn",
      "name": "触发源设备sn",
      "type": "string"
    },
    {
      "id": "priority",
      "name": "优先级",
      "type": "integer",
      "specs": {
        "min": 0,
        "max": 99
      }
    },
    {
      "id": "timeout",
      "name": "规则动作执行超时时间",
      "type": "integer",
      "specs": {
        "min": -1
      }
    },
    {
      "id": "retryCount",
      "name": "执行动作失败重试次数",
      "type": "integer",
      "specs": {
        "min": 0,
        "max": 10
      }
    },
    {
      "id": "conditions",
      "name": "执行条件，条件是与(并且)关系",
      "type": "array",
      "columnComplex": [
        {
          "id": "target",
          "name": "输入规则的事件参数，暂时不支持数组，结构体层级结构使用英文.作为分隔符",
          "type": "string"
        },
        {
          "id": "operator",
          "name": "判断符",
          "type": "string",
          "specs": {
            "optional": [
              {
                "value": "GT",
                "desc": "大于"
              },
              {
                "value": "GTE",
                "desc": "大于等于"
              },
              {
                "value": "LT",
                "desc": "小于"
              },
              {
                "value": "LTE",
                "desc": "小于等于"
              },
              {
                "value": "E",
                "desc": "等于"
              },
              {
                "value": "NE",
                "desc": "不等于"
              },
              {
                "value": "CONTAIN",
                "desc": "包含"
              },
              {
                "value": "NOT_CONTAIN",
                "desc": "不包含"
              }
            ]
          }
        },
        {
          "id": "comparison",
          "name": "比较值",
          "type": "string"
        }
      ]
    },
    {
      "id": "actions",
      "name": "执行动作",
      "type": "array",
      "columnComplex": [
        {
          "id": "type",
          "name": "执行动作类型",
          "type": "string",
          "specs": {
            "optional": [
              {
                "value": "CALL_SERVICE",
                "desc": "CALL_SERVICE"
              },
              {
                "value": "EVENT",
                "desc": "EVENT"
              },
              {
                "value": "WORKFORM",
                "desc": "WORKFORM"
              },
              {
                "value": "MESSAGE",
                "desc": "MESSAGE"
              },
              {
                "value": "EMAIL",
                "desc": "EMAIL"
              },
              {
                "value": "SLEEP",
                "desc": "SLEEP"
              }
            ]
          }
        },
        {
          "id": "productIdentifier",
          "name": "执行动作的产品标识符",
          "type": "string"
        },
        {
          "id": "devSn",
          "name": "执行动作的设备sn",
          "type": "string"
        },
        {
          "id": "params",
          "name": "执行动作参数，因动作类型不同而异，详情请查看规则json描述文档",
          "type": "string"
        }
      ]
    },
    {
      "id": "effectiveDate",
      "name": "规则有效时间周期",
      "type": "struct",
      "struct": [
        {
          "id": "startDate",
          "name": "开始日期",
          "format": "date",
          "type": "string"
        },
        {
          "id": "endDate",
          "name": "结束日期",
          "format": "date",
          "type": "string"
        },
        {
          "id": "startTime",
          "name": "开始时间",
          "format": "time",
          "type": "string"
        },
        {
          "id": "endTime",
          "name": "结束时间",
          "format": "time",
          "type": "string"
        }
      ]
    },
    {
      "id": "eventIdentifier",
      "name": "事件标识符",
      "type": "string"
    }
  ],
  "output": []
}
```

### 删除策略

```json

{
  "id": "deleteRuleById",
  "name": "删除策略",
  "parameters": [
    {
      "id": "id",
      "name": "规则ID",
      "type": "string"
    }
  ],
  "output": []
}
```

### 获取所有策略

```json
{
  "id": "getAllRules",
  "name": "获取所有策略",
  "parameters": [
    {
      "id": "page",
      "name": "分页参数",
      "type": "struct",
      "struct": [
        {
          "id": "offset",
          "name": "偏移",
          "type": "integer"
        },
        {
          "id": "paged",
          "name": "是否分页",
          "type": "integer",
          "specs": {
            "optional": [
              {
                "value": 0,
                "desc": "不分页"
              },
              {
                "value": "1",
                "desc": "分页"
              }
            ]
          }
        },
        {
          "id": "pageSize",
          "name": "每页大小",
          "type": "integer"
        },
        {
          "id": "pageNum",
          "name": "页号",
          "type": "integer"
        }
      ]
    },
    {
      "id": "name",
      "name": "规则名称",
      "type": "string"
    },
    {
      "id": "description",
      "name": "规则描述",
      "type": "string"
    },
    {
      "id": "priority",
      "name": "优先级",
      "type": "integer",
      "specs": {
        "min": 0,
        "max": 99
      }
    },
    {
      "id": "productIdentifier",
      "name": "触发源产品标识符",
      "type": "string"
    },
    {
      "id": "devSn",
      "name": "触发源设备sn",
      "type": "string"
    },
    {
      "id": "active",
      "name": "停用启用状态",
      "type": "string",
      "specs": {
        "optional": [
          {
            "value": "Y",
            "desc": "启用"
          },
          {
            "value": "N",
            "desc": "停用"
          }
        ]
      }
    }
  ],
  "output": [
    {
      "id": "data",
      "name": "规则列表",
      "type": "array",
      "columnComplex": [
        {
          "id": "id",
          "name": "规则ID",
          "type": "string"
        },
        {
          "id": "name",
          "name": "规则名字",
          "type": "string"
        },
        {
          "id": "active",
          "name": "停用启用状态",
          "type": "string",
          "specs": {
            "optional": [
              {
                "value": "Y",
                "desc": "启用"
              },
              {
                "value": "N",
                "desc": "停用"
              }
            ]
          }
        },
        {
          "id": "description",
          "name": "规则描述",
          "type": "string"
        },
        {
          "id": "productIdentifier",
          "name": "触发源产品标识符",
          "type": "string"
        },
        {
          "id": "devSn",
          "name": "触发源设备sn",
          "type": "string"
        },
        {
          "id": "priority",
          "name": "优先级",
          "type": "integer",
          "specs": {
            "min": 0,
            "max": 99
          }
        },
        {
          "id": "timeout",
          "name": "规则动作执行超时时间",
          "type": "integer",
          "specs": {
            "min": -1
          }
        },
        {
          "id": "retryCount",
          "name": "执行动作失败重试次数",
          "type": "integer",
          "specs": {
            "min": 0,
            "max": 10
          }
        },
        {
          "id": "conditions",
          "name": "执行条件，条件是与(并且)关系",
          "type": "array",
          "columnComplex": [
            {
              "id": "target",
              "name": "输入规则的事件参数，暂时不支持数组，结构体层级结构使用英文.作为分隔符",
              "type": "string"
            },
            {
              "id": "operator",
              "name": "判断符",
              "type": "string",
              "specs": {
                "optional": [
                  {
                    "value": "GT",
                    "desc": "大于"
                  },
                  {
                    "value": "GTE",
                    "desc": "大于等于"
                  },
                  {
                    "value": "LT",
                    "desc": "小于"
                  },
                  {
                    "value": "LTE",
                    "desc": "小于等于"
                  },
                  {
                    "value": "E",
                    "desc": "等于"
                  },
                  {
                    "value": "NE",
                    "desc": "不等于"
                  },
                  {
                    "value": "CONTAIN",
                    "desc": "包含"
                  },
                  {
                    "value": "NOT_CONTAIN",
                    "desc": "不包含"
                  }
                ]
              }
            },
            {
              "id": "comparison",
              "name": "比较值",
              "type": "string"
            }
          ]
        },
        {
          "id": "actions",
          "name": "执行动作",
          "type": "array",
          "columnComplex": [
            {
              "id": "type",
              "name": "执行动作类型",
              "type": "string",
              "specs": {
                "optional": [
                  {
                    "value": "CALL_SERVICE",
                    "desc": "CALL_SERVICE"
                  },
                  {
                    "value": "EVENT",
                    "desc": "EVENT"
                  },
                  {
                    "value": "WORKFORM",
                    "desc": "WORKFORM"
                  },
                  {
                    "value": "MESSAGE",
                    "desc": "MESSAGE"
                  },
                  {
                    "value": "EMAIL",
                    "desc": "EMAIL"
                  },
                  {
                    "value": "SLEEP",
                    "desc": "SLEEP"
                  }
                ]
              }
            },
            {
              "id": "productIdentifier",
              "name": "执行动作的产品标识符",
              "type": "string"
            },
            {
              "id": "devSn",
              "name": "执行动作的设备sn",
              "type": "string"
            },
            {
              "id": "params",
              "name": "执行动作参数，因动作类型不同而异，详情请查看规则json描述文档",
              "type": "string"
            }
          ]
        },
        {
          "id": "effectiveDate",
          "name": "规则有效时间周期",
          "type": "struct",
          "struct": [
            {
              "id": "startDate",
              "name": "开始日期",
              "format": "date",
              "type": "string"
            },
            {
              "id": "endDate",
              "name": "结束日期",
              "format": "date",
              "type": "string"
            },
            {
              "id": "startTime",
              "name": "开始时间",
              "format": "time",
              "type": "string"
            },
            {
              "id": "endTime",
              "name": "结束时间",
              "format": "time",
              "type": "string"
            }
          ]
        },
        {
          "id": "eventIdentifier",
          "name": "事件标识符",
          "type": "string"
        }
      ]
    },
    {
      "id": "page",
      "name": "分页详情",
      "type": "struct",
      "struct": [
        {
          "id": "size",
          "name": "每页大小",
          "type": "integer"
        },
        {
          "id": "page",
          "name": "页号",
          "type": "integer"
        },
        {
          "id": "total",
          "name": "总条数",
          "type": "integer"
        }
      ]
    }
  ]
}
```

### 获取策略详情

```json
{
  "id": "getRule",
  "name": "获取策略详情",
  "parameters": [
    {
      "id": "id",
      "name": "规则ID",
      "type": "string"
    }
  ],
  "output": [
    {
      "id": "id",
      "name": "规则ID",
      "type": "string"
    },
    {
      "id": "name",
      "name": "规则名字",
      "type": "string"
    },
    {
      "id": "active",
      "name": "停用启用状态",
      "type": "string",
      "specs": {
        "optional": [
          {
            "value": "Y",
            "desc": "启用"
          },
          {
            "value": "N",
            "desc": "停用"
          }
        ]
      }
    },
    {
      "id": "description",
      "name": "规则描述",
      "type": "string"
    },
    {
      "id": "productIdentifier",
      "name": "触发源产品标识符",
      "type": "string"
    },
    {
      "id": "devSn",
      "name": "触发源设备sn",
      "type": "string"
    },
    {
      "id": "priority",
      "name": "优先级",
      "type": "integer",
      "specs": {
        "min": 0,
        "max": 99
      }
    },
    {
      "id": "timeout",
      "name": "规则动作执行超时时间",
      "type": "integer",
      "specs": {
        "min": -1
      }
    },
    {
      "id": "retryCount",
      "name": "执行动作失败重试次数",
      "type": "integer",
      "specs": {
        "min": 0,
        "max": 10
      }
    },
    {
      "id": "conditions",
      "name": "执行条件，条件是与(并且)关系",
      "type": "array",
      "columnComplex": [
        {
          "id": "target",
          "name": "输入规则的事件参数，暂时不支持数组，结构体层级结构使用英文.作为分隔符",
          "type": "string"
        },
        {
          "id": "operator",
          "name": "判断符",
          "type": "string",
          "specs": {
            "optional": [
              {
                "value": "GT",
                "desc": "大于"
              },
              {
                "value": "GTE",
                "desc": "大于等于"
              },
              {
                "value": "LT",
                "desc": "小于"
              },
              {
                "value": "LTE",
                "desc": "小于等于"
              },
              {
                "value": "E",
                "desc": "等于"
              },
              {
                "value": "NE",
                "desc": "不等于"
              },
              {
                "value": "CONTAIN",
                "desc": "包含"
              },
              {
                "value": "NOT_CONTAIN",
                "desc": "不包含"
              }
            ]
          }
        },
        {
          "id": "comparison",
          "name": "比较值",
          "type": "string"
        }
      ]
    },
    {
      "id": "actions",
      "name": "执行动作",
      "type": "array",
      "columnComplex": [
        {
          "id": "type",
          "name": "执行动作类型",
          "type": "string",
          "specs": {
            "optional": [
              {
                "value": "CALL_SERVICE",
                "desc": "CALL_SERVICE"
              },
              {
                "value": "EVENT",
                "desc": "EVENT"
              },
              {
                "value": "WORKFORM",
                "desc": "WORKFORM"
              },
              {
                "value": "MESSAGE",
                "desc": "MESSAGE"
              },
              {
                "value": "EMAIL",
                "desc": "EMAIL"
              },
              {
                "value": "SLEEP",
                "desc": "SLEEP"
              }
            ]
          }
        },
        {
          "id": "productIdentifier",
          "name": "执行动作的产品标识符",
          "type": "string"
        },
        {
          "id": "devSn",
          "name": "执行动作的设备sn",
          "type": "string"
        },
        {
          "id": "params",
          "name": "执行动作参数，因动作类型不同而异，详情请查看规则json描述文档",
          "type": "string"
        }
      ]
    },
    {
      "id": "effectiveDate",
      "name": "规则有效日期",
      "type": "struct",
      "struct": [
        {
          "id": "startDate",
          "name": "开始日期",
          "format": "date",
          "type": "string"
        },
        {
          "id": "endDate",
          "name": "结束日期",
          "format": "date",
          "type": "string"
        },
        {
          "id": "startTime",
          "name": "开始时间",
          "format": "time",
          "type": "string"
        },
        {
          "id": "endTime",
          "name": "结束时间",
          "format": "time",
          "type": "string"
        }
      ]
    },
    {
      "id": "eventIdentifier",
      "name": "事件标识符",
      "type": "string"
    }
  ]
}
```

### 启用策略

```json
{
  "id": "enableRule",
  "name": "启用策略",
  "parameters": [
    {
      "id": "id",
      "name": "规则ID",
      "type": "string"
    }
  ],
  "output": []
}
```

### 停用策略

```json
{
  "id": "disableRule",
  "name": "停用策略",
  "parameters": [
    {
      "id": "id",
      "name": "规则ID",
      "type": "string"
    }
  ],
  "output": []
}
```

### 外部事件联动

```json
{
  "id": "outsideEvent",
  "name": "外部联动事件",
  "parameters": [
    {
      "id": "eventIdentifier",
      "name": "事件标识符",
      "type": "string"
    },
    {
      "id": "productIdentifier",
      "name": "产品标识符",
      "type": "string"
    },
    {
      "id": "devSn",
      "name": "设备标识符",
      "type": "string"
    },
    {
      "id": "timestamp",
      "name": "时间戳",
      "type": "integer"
    },
    {
      "id": "params",
      "name": "规则参数",
      "type": "struct",
      "struct": [
        {
          "id": "serviceIdentifier",
          "name": "服务标识符",
          "type": "string"
        },
        {
          "id": "serviceParams",
          "name": "服务参数",
          "type": "string"
        },
        {
          "id": "timeout",
          "name": "生效时间",
          "type": "integer"
        },
        {
          "id": "priority",
          "name": "优先级",
          "type": "integer",
          "specs": {
            "min": 1,
            "max": 99
          }
        },
        {
          "id": "retryCount",
          "name": "重试次数",
          "type": "integer"
        }
      ]
    }
  ],
  "output": []
}
```

### 规则统计

```json
{
  "id": "ruleCount",
  "name": "规则统计",
  "parameters": [
    {
      "id": "running",
      "name": "执行状态",
      "type": "string"
    },
    {
      "id": "active",
      "name": "停用启用状态",
      "type": "string"
    },
    {
      "id": "eventIdentifier",
      "name": "规则触发事件",
      "type": "string"
    },
    {
      "id": "productIdentifier",
      "name": "触发源的产品标识符",
      "type": "string"
    },
    {
      "id": "devSn",
      "name": "触发源设备sn",
      "type": "string"
    }
  ],
  "output": [
    {
      "id": "count",
      "name": "符合筛选条件的规则数量",
      "type": "integer"
    },
    {
      "id": "total",
      "name": "规则总数量",
      "type": "integer"
    }
  ]
}
```


## 规则执行

### 定时策略
- 规则引擎在收到设备定时开始事件时,会根据当前设备执行策略的情况进行决策.
    - 如果当前设备所执行的联动策略的优先级大于等于定时策略的优先级(定时策略优先级固定为50)，定时开始事件被忽略.
    - 如果当前设备没有执行联动策略或执行的联动策略的优先级小于定时策略的优先级(50),则低优先级策略被打断,执行设备定时策略.
- 规则引擎在收到定时默认事件时,会根据当前设备执行策略的情况进行决策.
    - 如果当前设备没有执行联动策略,则执行默认定时策略,使设备恢复默认.
    - 规则引擎在收到定时默认事件时,如果当前设备有执行联动策略,则定时默认事件被忽略.

### 手动控制
- 规则引擎在收到设备手动控制事件时,会打断当前设备所执行的策略,执行手动控制策略所描述的服务动作,手动控制策略的优先级最高(100)
,执行时间为5min

### 联动策略
- 规则引擎在收到设备根据自身知识产生的事件时,会进行规则匹配,如果匹配到联动策略,则进行规则优先级决策.打断当前设备所执行的低优先级策略或忽略执行当前匹配到的联动策略。

### 外部事件联动
- 规则引擎在收到平台的外部事件时,会根据事件参数中的优先级,进行规则优先级决策.打断当前设备所执行的低优先级策略或忽略执行外部事件所匹配到的联动策略。
